In [107]:
import sys
import os

package_path = os.path.abspath('coeqwalpackage') 
if package_path not in sys.path:
    sys.path.insert(0, package_path)

import cqwlutils as cu
import plotting as pu


Load Historical Data

In [108]:
import pandas as pd

file_path = "/Users/rain/Desktop/DSP/coeqwal/Historical_data/20250506_MonthlyResData_CDEC-SHA.csv"
df_raw = pd.read_csv(file_path, header=None)

start_row = df_raw[df_raw.apply(lambda row: row.astype(str).str.contains('RESERVOIR STORAGE').any(), axis=1)].index[0]
df = pd.read_csv(file_path, skiprows=start_row)
print(df.head())


     SnsrType LAKE COMPUTED AF EVAPORATION LAKE COMPUTED AF EVAPORATION.1  \
0     SnsrNum                           68                             68   
1       Units                           AF                             AF   
2       Value                        VALUE                      DATA_FLAG   
3        DATE                          NaN                            NaN   
4  1953-10-01                          NaN                                  

  ACCUMULATED PRECIPITATION ACCUMULATED PRECIPITATION.1 REVISED PRECIPITATION  \
0                         2                           2                    80   
1                    INCHES                      INCHES                INCHES   
2                     VALUE                   DATA_FLAG                 VALUE   
3                       NaN                         NaN                   NaN   
4                       NaN                                               NaN   

  REVISED PRECIPITATION.1 RESERVOIR STORAGE RESERV

In [109]:
import pandas as pd

def extract_reservoir_storage_from_file(file_path):
    """
    Extracts and cleans historical reservoir storage data from a formatted CSV file.
    
    Looks for a column that contains 'RESERVOIR STORAGE' in row 3 and 'VALUE' in row 6.
    
    Parameters:
        file_path (str): Path to the CSV file
    
    Returns:
        df_clean (pd.DataFrame): Cleaned DataFrame with columns 'DATE' and 'RESERVOIR STORAGE'
    """
    df_raw = pd.read_csv(file_path, header=None)

    target_col_index = None
    for col in df_raw.columns:
        if (
            str(df_raw.iloc[3, col]).strip() == 'RESERVOIR STORAGE' and
            str(df_raw.iloc[6, col]).strip() == 'VALUE'
        ):
            target_col_index = col
            break

    if target_col_index is None:
        raise ValueError("No column with 'RESERVOIR STORAGE' and 'VALUE' found in expected rows.")

    df_data = df_raw.iloc[7:].copy()
    df_data = df_data[[0, target_col_index]]
    df_data.columns = ['DATE', 'RESERVOIR STORAGE']

    df_data['DATE'] = pd.to_datetime(df_data['DATE'], errors='coerce')
    df_data['RESERVOIR STORAGE'] = pd.to_numeric(df_data['RESERVOIR STORAGE'], errors='coerce')
    df_clean = df_data.dropna()

    return df_clean


In [110]:
file_path = "/Users/rain/Desktop/DSP/coeqwal/Historical_data/20250506_MonthlyResData_CDEC-SHA.csv"
df_clean = extract_reservoir_storage_from_file(file_path)
april_df, thresholds = compute_april_storage_thresholds(df_clean)

/var/folders/q2/lk8v_g9j7pd3cppdtbty00240000gn/T/ipykernel_33735/1091298288.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_data['DATE'] = pd.to_datetime(df_data['DATE'], errors='coerce')


In [111]:
print(df_data.tail(5))  # shows last 5 rows


          DATE  RESERVOIR STORAGE
862 2024-12-01           3477.400
863 2025-01-01           3492.457
864 2025-02-01           3561.400
865 2025-03-01           4067.273
866 2025-04-01           4396.007


Calculate Percentiles

In [112]:
import pandas as pd

def compute_april_storage_thresholds(df, percentiles=(0.25, 0.5, 0.9)):
    """
    Computes specified percentile thresholds for end-of-April reservoir storage (in TAF).

    Parameters:
        df (pd.DataFrame): Input DataFrame with 'DATE' and 'RESERVOIR STORAGE' columns in AF
        percentiles (tuple): A tuple of three percentiles (e.g., (0.25, 0.5, 0.9))

    Returns:
        april_df (pd.DataFrame): End-of-April values per year
        thresholds_df (pd.DataFrame): DataFrame with Percentile and Threshold (TAF) columns
    """
    df = df.copy()

    df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')
    df['RESERVOIR STORAGE'] = pd.to_numeric(df['RESERVOIR STORAGE'], errors='coerce') / 1000  # convert AF to TAF
    df = df.dropna()

    # Filter for April and group by year (last value of April)
    df_april = df[df['DATE'].dt.month == 4]
    df_april = df_april.groupby(df_april['DATE'].dt.year).last()

    values = df_april['RESERVOIR STORAGE']
    threshold_values = [values.quantile(p) for p in percentiles]
    labels = [f"{int(p * 100)}th" for p in percentiles]

    thresholds_df = pd.DataFrame({
        'Percentile': labels,
        'Threshold (TAF)': threshold_values
    })

    return df_april, thresholds_df


In [113]:
april_df, thresholds = compute_april_storage_thresholds(df_clean)
print("Thresholds (TAF):", thresholds)
print(april_df.head())


Thresholds (TAF):   Percentile  Threshold (TAF)
0       25th        3728.4715
1       50th        4194.7660
2       90th        4431.9000
           DATE  RESERVOIR STORAGE
DATE                              
1954 1954-04-01             4431.0
1955 1955-04-01             3678.8
1957 1957-04-01             4431.3
1958 1958-04-01             4415.5
1959 1959-04-01             3948.7


Load CalSim Data

In [120]:
import pandas as pd
import re


CtrlFile = 'CalSim3DataExtractionInitFile_v3.xlsx'
CtrlTab = 'Init'

ScenarioListFile, ScenarioListTab, ScenarioListPath, DVDssNamesOutPath, SVDssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, DVDssMin, DVDssMax, SVDssMin, SVDssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax = cu.read_init_file(CtrlFile, CtrlTab)


df, dss_names = read_in_df(ConvertDataOutPath,DVDssNamesOutPath)
df.columns = ['_'.join(map(str, col)) if isinstance(col, tuple) else col for col in df.columns]

scenario_codes = sorted(set(re.findall(r's\d{4}', '_'.join(df.columns))))
scenario_codes = [s for s in scenario_codes if s != 's0017']

shasta_cols = [
    col for col in df.columns
    if 'SHSTA' in col and any(s in col for s in scenario_codes)
]



In [121]:
import re
import pandas as pd

def extract_variable_by_scenario(df, variable, value_type='STORAGE'):
    """
    Extracts CalSim variable columns (e.g., S_SHSTA) for all scenarios s0001–s0018 (excluding s0017),
    filtering by value type (e.g., 'STORAGE', not 'STORAGE-LEVEL').

    Parameters:
        df (pd.DataFrame): Wide-format CalSim3 output DataFrame with datetime index
        variable (str): The core variable name to match (e.g., 'S_SHSTA')
        value_type (str): Filter substring that must appear in column (default: 'STORAGE')

    Returns:
        pd.DataFrame: A DataFrame with one column per scenario + 'DATE'
    """
    df.columns = ['_'.join(map(str, col)) if isinstance(col, tuple) else col for col in df.columns]

    scenario_codes = [f's{str(i).zfill(4)}' for i in range(1, 19) if i != 17]

    matched_cols = [
        col for col in df.columns
        if col.startswith(f'CALSIM_{variable}_') and
           value_type in col and
           not f'{value_type}-' in col and
           any(s in col for s in scenario_codes)
    ]

    if not matched_cols:
        print(f"No matches found for variable '{variable}' with value type '{value_type}'")
        return pd.DataFrame()

    var_df = df[matched_cols].copy()
    var_df.columns = [re.search(r's\d{4}', col).group(0) for col in matched_cols]
    var_df['DATE'] = df.index

    return var_df


In [116]:
shasta_df = extract_variable_by_scenario(df, 'S_SHSTA')
print(shasta_df.head())


                s0001      s0002      s0003      s0004      s0005      s0006  \
1921-10-31  3057.8250  3055.7650  3052.4030  3049.2104  3045.7935  2739.6995   
1921-11-30  2908.5920  2901.6628  2907.8103  2880.3591  2902.9326  2642.8274   
1921-12-31  3076.5317  3074.3784  3153.1946  3126.2263  3141.3110  2815.1300   
1922-01-31  3204.3298  3165.3650  3258.3555  3224.1177  3231.2770  2947.1340   
1922-02-28  3585.4524  3590.3950  3786.1345  3742.7375  3732.1300  3334.8300   

                s0007      s0008      s0009      s0010      s0011      s0012  \
1921-10-31  2739.6995  2739.6995  2739.6995  2739.6995  3055.7650  3056.3748   
1921-11-30  2642.8274  2642.8274  2642.8274  2642.8274  2901.6660  2902.6270   
1921-12-31  2815.1300  2815.1300  2815.1300  2815.1300  3074.3816  3075.3418   
1922-01-31  2947.1340  2947.1340  2947.1340  2947.1340  3165.3684  3166.3280   
1922-02-28  3334.8300  3334.8300  3334.8300  3334.8300  3590.3984  3591.3574   

                s0013      s0014      

Overall Function

In [117]:
import pandas as pd
import numpy as np
import re

def load_historical_storage_csv(filepath):
    """
    Loads historical SHA reservoir data from raw CSV with no headers.
    Returns a cleaned DataFrame with 'DATE' and 'RESERVOIR STORAGE' columns (AF).
    """
    df_raw = pd.read_csv(filepath, header=None)

    target_col_index = None
    for col in df_raw.columns:
        if (
            str(df_raw.iloc[3, col]).strip() == 'RESERVOIR STORAGE' and
            str(df_raw.iloc[6, col]).strip() == 'VALUE'
        ):
            target_col_index = col
            break

    if target_col_index is None:
        raise ValueError("Could not find RESERVOIR STORAGE column in historical data.")

    df_data = df_raw.iloc[7:].copy()
    df_data = df_data[[0, target_col_index]]
    df_data.columns = ['DATE', 'RESERVOIR STORAGE']
    df_data['DATE'] = pd.to_datetime(df_data['DATE'], errors='coerce')
    df_data['RESERVOIR STORAGE'] = pd.to_numeric(df_data['RESERVOIR STORAGE'], errors='coerce')
    df_data = df_data.dropna()

    return df_data

def extract_historical_thresholds(df_hist, month=4):
    """
    Calculate 25th, 50th, and 90th percentiles from historical storage data.
    Converts AF to TAF.
    """
    df = df_hist.copy()
    df['RESERVOIR STORAGE'] = df['RESERVOIR STORAGE'] / 1000  # Convert AF to TAF
    df = df[df['DATE'].dt.month == month]
    df = df[df['DATE'].dt.is_month_end]  # pick end-of-month values

    storage_values = df['RESERVOIR STORAGE'].dropna()
    p25 = np.percentile(storage_values, 25)
    p50 = np.percentile(storage_values, 50)
    p90 = np.percentile(storage_values, 90)

    return pd.DataFrame({
        'Percentile': ['25th', '50th', '90th'],
        'Threshold (TAF)': [p25, p50, p90]
    })

def extract_historical_thresholds(df_hist, month=5, day=1):
    """
    Calculate 25th, 50th, and 90th percentiles from historical storage data.
    Uses AF → TAF conversion. Filters for values on specified month/day 
    (e.g., May 1 to represent end-of-April).
    """
    df = df_hist.copy()
    df['RESERVOIR STORAGE'] = df['RESERVOIR STORAGE'] / 1000  # Convert AF to TAF

    df = df[(df['DATE'].dt.month == month) & (df['DATE'].dt.day == day)]

    storage_values = df['RESERVOIR STORAGE'].dropna()

    if storage_values.empty:
        raise ValueError(f"No historical data found for {month:02d}-{day:02d} (e.g., May 1st for end-of-April).")

    p25 = np.percentile(storage_values, 25)
    p50 = np.percentile(storage_values, 50)
    p90 = np.percentile(storage_values, 90)

    return pd.DataFrame({
        'Percentile': ['25th', '50th', '90th'],
        'Threshold (TAF)': [p25, p50, p90]
    })


def assign_tiers_from_calsim(var_df, thresholds_df, month=4):
    """
    Assign Tier 1–4 to each scenario based on April-end modeled storage vs historical percentiles.
    """
    p25 = thresholds_df.loc[thresholds_df['Percentile'] == '25th', 'Threshold (TAF)'].values[0]
    p50 = thresholds_df.loc[thresholds_df['Percentile'] == '50th', 'Threshold (TAF)'].values[0]
    p90 = thresholds_df.loc[thresholds_df['Percentile'] == '90th', 'Threshold (TAF)'].values[0]

    results = []

    for scenario in var_df.columns:
        if scenario == 'DATE':
            continue

        df_scenario = var_df[['DATE', scenario]].dropna()
        df_scenario = df_scenario[df_scenario['DATE'].dt.month == month]
        df_scenario = df_scenario[df_scenario['DATE'].dt.is_month_end]

        total_years = df_scenario['DATE'].dt.year.nunique()
        values = df_scenario[scenario]

        top = (values > p90).sum() / total_years
        mid = ((values > p50) & (values <= p90)).sum() / total_years
        low = ((values > p25) & (values <= p50)).sum() / total_years
        bot = (values <= p25).sum() / total_years

        if top >= 0.9:  #This can be changed based on requirement
            tier = 1
        elif (top + mid) >= 0.9:
            tier = 2
        elif (top + mid) >= 0.3:
            tier = 3
        else:
            tier = 4

        results.append({
            'Scenario': scenario,
            'Top': round(top, 2),
            'Mid': round(mid, 2),
            'Low': round(low, 2),
            'Bot': round(bot, 2),
            'Tier': tier
        })

    return pd.DataFrame(results)


In [118]:
# Load and clean historical file
hist_df = load_historical_storage_csv("/Users/rain/Desktop/DSP/coeqwal/Historical_data/20250506_MonthlyResData_CDEC-SHA.csv")

# Compute percentiles from historical
thresholds_df = extract_historical_thresholds(hist_df, month=4)

# Extract CalSim output for S_SHSTA
shasta_df = extract_variable_by_scenario(df, 'S_SHSTA', value_type='STORAGE')

# Assign tiers
tier_df = assign_tiers_from_calsim(shasta_df, thresholds_df, month=4)

print(tier_df)


   Scenario   Top   Mid   Low   Bot  Tier
0     s0001  0.35  0.17  0.31  0.17     3
1     s0002  0.24  0.24  0.29  0.23     3
2     s0003  0.14  0.24  0.28  0.34     3
3     s0004  0.13  0.20  0.29  0.38     3
4     s0005  0.12  0.18  0.25  0.45     3
5     s0006  0.35  0.20  0.27  0.18     3
6     s0007  0.35  0.20  0.28  0.17     3
7     s0008  0.35  0.20  0.28  0.17     3
8     s0009  0.35  0.20  0.27  0.18     3
9     s0010  0.35  0.20  0.27  0.18     3
10    s0011  0.24  0.24  0.32  0.20     3
11    s0012  0.24  0.24  0.31  0.21     3
12    s0013  0.13  0.20  0.30  0.37     3
13    s0014  0.13  0.21  0.33  0.33     3
14    s0015  0.24  0.24  0.33  0.19     3
15    s0016  0.13  0.21  0.33  0.33     3
16    s0018  0.01  0.07  0.21  0.71     4


/var/folders/q2/lk8v_g9j7pd3cppdtbty00240000gn/T/ipykernel_33735/3645769524.py:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_data['DATE'] = pd.to_datetime(df_data['DATE'], errors='coerce')
